In [6]:
import rasterio
from rasterio.enums import Resampling
from rasterio.shutil import copy
from rio_cogeo import cog_validate

import numpy as np

#### Convert GeoTiff to Cloud-Optimized GeoTiff

The conversion can also be implemented using `rio-cogeo 5.4.1 `, but this is a simple conversion with `rasterio` and `GDAL` in the background.  
Requirements:  
    `GDAL >= 3.1`  
    `rasterio >= 1.2`  
An example of rio-cogeo can be found here: https://guide.cloudnativegeo.org/cloud-optimized-geotiffs/writing-cogs-in-python.html

In [2]:
def nearest_power_of_two(n):
    """
    Returns the largest power of two <= n (if n is a power of two), with the maximum possible value being 512.
    """
    power = 2**(n.bit_length() - 1)
    return min(power, 512)

def choose_predictor(dtype):
    """
    Returns predictor value based on data type.
    """
    if dtype.startswith('float'):
        predictor = 3
    else:
        predictor = 2

    return predictor

In [5]:
# Path to your input raster file
input_path = "/home/data/S2A_MSIL1C_20241105T091151_N0511_R050_T34SGG_20241105T100127_reprojected_mskchla_linear_gapfilled.tif"
output_path = "/home/data/S2A_MSIL1C_20241105T091151_N0511_R050_T34SGG_20241105T100127_reprojected_mskchla_linear_gapfilled_COG.tif"

with rasterio.open(input_path) as src:
    profile = src.profile.copy()

    #### Build Pyramids (i.e., overviews)
    factors = [2, 4, 8, 16] # If the output image size is small, Pyramids may not be built, cause GDAL thinks they are not needed.

    #### Define block sizes to be powers of 2, but less than 512
    width, height = src.width, src.height
    max_blocksize = 2**9 # 512
    blockxsize = nearest_power_of_two(min(width, max_blocksize))
    blockysize = nearest_power_of_two(min(height, max_blocksize))

    #### Select predictor value
    predictor = choose_predictor(src.dtypes[0])  # 2 for integers, 3 for floats
    
    # Use rasterio.shutil.copy to save as COG
    copy(
        src,
        output_path,
        driver="COG", # this works only with rasterio >= 1.2 and GDAL >= 3.1
        compress="deflate",
        overview_resampling="nearest", # Resampling.nearest,
        overview_levels=factors,
        blockxsize=blockxsize,
        blockysize=blockysize,
        interleave="band",
        BIGTIFF="IF_SAFER", # depends on the image size (if it's > 4gb)
        predictor=predictor,
        tiled=True
    )    

#### Validate COG file

The following can be used for validation of the output COG and can be included in a report generated per product.

In [8]:
cog_validate(output_path), cog_validate(input_path)

The following warnings were found:
- The file is greater than 512xH or 512xW, it is recommended to include internal overviews

The following errors were found:
- The file is greater than 512xH or 512xW, but is not tiled


((True, [], []),
 (False,
  ['The file is greater than 512xH or 512xW, but is not tiled'],
  ['The file is greater than 512xH or 512xW, it is recommended to include internal overviews']))